In [34]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
pd.set_option('display.max_columns', None)

In [35]:
### prepare data
df_t = pd.read_excel('data.xlsx')
df_t.columns = df_t.columns.str.strip()
# random value
# new_df = pd.DataFrame({col: np.random.choice(df[col], size=500) for col in df.columns})

In [36]:
df = df_t
df = df.iloc[57:]

### clean data

In [37]:
### 2.1 ในเดือนที่ผ่านมาท่านทานบุฟเฟต์ชาบูกี่ครั้ง
condition = [df['2.1 ในเดือนที่ผ่านมาท่านทานบุฟเฟต์ชาบูกี่ครั้ง'].isin(['1-3 ครั้ง','1 - 4 ครั้ง']),df['2.1 ในเดือนที่ผ่านมาท่านทานบุฟเฟต์ชาบูกี่ครั้ง'].isin(['ไม่ได้รับประทานเลย'])]
df['2.1 ในเดือนที่ผ่านมาท่านทานบุฟเฟต์ชาบูกี่ครั้ง'] =  np.select(condition,['1 - 4 ครั้ง','ไม่ได้รับประทานเลย'] , default='มากกว่า 4 ครั้ง')
df['2.1 ในเดือนที่ผ่านมาท่านทานบุฟเฟต์ชาบูกี่ครั้ง'].unique()

### 1.4 อายุ
condition = [df['1.4 อายุ'].isin(['21-30 ปี','21 - 30 ปี']),df['1.4 อายุ'].isin(['31-40 ปี','31 - 40 ปี']),df['1.4 อายุ'].isin(['41 - 50 ปี','51 ปีขึ้นไป'])]
df['1.4 อายุ'] =  np.select(condition,['21 - 30 ปี','31 - 40 ปี','40 ปีขึ้นไป'],default='ต่ำกว่า 20 ปี')
df['1.4 อายุ'].unique()

#1.6 อาชีพ
condition = [df['1.6 อาชีพ'].isin(['ข้าราชการ/รัฐวิสาหกิจ','พนักงานของรัฐ','พนักงานรัฐ','ลูกจ้างของหน่วยงานรัฐ','ลูกจ้างชั่วคราว ภาครัฐ','ลูกจ้างหน่วยงานรัฐ','ลูกจ้างหน่วยงานราชการ','ลูกจ้างในหน่วยงานราชการ '])
             ,df['1.6 อาชีพ'].isin(['พนักงานบริษัทเอกชน'])]
df['1.6 อาชีพ'] =  np.select(condition,['ข้าราชการ/รัฐวิสาหกิจ','พนักงานบริษัทเอกชน'],default='อื่น')
df['1.6 อาชีพ'].unique()

### หัวข้อ 3
df['3.1 product_hat'] = (df['3.1 ด้านผลิตภัณฑ์ (product)  [อาหารมีความหลากลาย]']+df['3.1 ด้านผลิตภัณฑ์ (product)  [รสชาติของอาหาร]']+df['3.1 ด้านผลิตภัณฑ์ (product)  [ความสะอาดของอาหาร]'])/3
df['3.2 Price_hat'] = (df['3.2 ด้านราคา (price)  [ราคาที่เหมาะสมกับคุณภาพ]']+df['3.2 ด้านราคา (price)  [ราคาของอาหารเหมาะสมเมื่อเทียบกับร้านอื่น]'])/2
df['3.3 Place_hat'] = (df['3.3 ด้านช่องทางการจัดจำหน่าย (Place)  [เวลาเปิด-ปิดของร้านในการให้บริการ]']+df['3.3 ด้านช่องทางการจัดจำหน่าย (Place)  [ความสะดวกของที่จอดรถ]']+df['3.3 ด้านช่องทางการจัดจำหน่าย (Place)  [ระยะทาง]'])/3
df['3.4 Promotion_hat'] = (df['3.4 ด้านโปรโมชั่น (promotion)  [ส่วนลดหรือโปรโมชั่นส่งเสริมการขายในช่วงเทศกาลต่าง ๆ เช่น เทศกาลขึ้นปีใหม่]']+df['3.4 ด้านโปรโมชั่น (promotion)  [การสื่อสารข้อมูลโปรโมชั่นผ่านช่องทางต่างๆ เช่น YouTube, Facebook, TikTok]']+df['3.4 ด้านโปรโมชั่น (promotion)  [การมอบสิทธิพิเศษกับสมาชิก]'])/3
df['3.5 People_hat'] = (df['3.5 ด้านบุคลากร (people) [พนักงานมี Service Mind ที่ดี]']+df['3.5 ด้านบุคลากร (people) [ความสะอาดในการแต่งกายของพนักงาน]']+df['3.5 ด้านบุคลากร (people) [จำนวนพนักงานมีเพียงพอในการให้บริการ]'])/3
df['3.6 process_hat'] = (df['3.6 ด้านกระบวนการ (process)  [การจัดลำดับคิวการให้บริการลูกค้า]']+df['3.6 ด้านกระบวนการ (process)  [กระบวนการสั่งเครื่องดื่มและอาหาร การรับคำสั่ง และการเสิร์ฟเครื่องดื่มและอาหารตามที่สั่ง]']+df['3.6 ด้านกระบวนการ (process)  [ระยะเวลาในการรออาหาร]'])/3

### 16. กลยุทธ์ส่งเสริมการขายแบบใดที่จะทำให้ท่านทานบุฟเฟต์ชาบู
condition = [df['6. กลยุทธ์ส่งเสริมการขายแบบใดที่จะทำให้ท่านทานบุฟเฟต์ชาบู'].isin(['ลดราคา']),df['6. กลยุทธ์ส่งเสริมการขายแบบใดที่จะทำให้ท่านทานบุฟเฟต์ชาบู'].isin(['กลยุทธ์ส่งเสริมการขายไม่มีผลต่อการตัดสินใจเลือกบริโภค'])]
df['6. กลยุทธ์ส่งเสริมการขายแบบใดที่จะทำให้ท่านทานบุฟเฟต์ชาบู'] =  np.select(condition,['ลดราคา','กลยุทธ์ส่งเสริมการขายไม่มีผลต่อการตัดสินใจเลือกบริโภค'] , default='อื่น ๆ')
df['6. กลยุทธ์ส่งเสริมการขายแบบใดที่จะทำให้ท่านทานบุฟเฟต์ชาบู'].unique()

C:\Users\chaiyaphonp\AppData\Local\Temp\ipykernel_2320\1135786901.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['2.1 ในเดือนที่ผ่านมาท่านทานบุฟเฟต์ชาบูกี่ครั้ง'] =  np.select(condition,['1 - 4 ครั้ง','ไม่ได้รับประทานเลย'] , default='มากกว่า 4 ครั้ง')
C:\Users\chaiyaphonp\AppData\Local\Temp\ipykernel_2320\1135786901.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['1.4 อายุ'] =  np.select(condition,['21 - 30 ปี','31 - 40 ปี','40 ปีขึ้นไป'],default='ต่ำกว่า 20 ปี')
C:\Users\chaiyaphonp\AppData\

array(['ลดราคา', 'กลยุทธ์ส่งเสริมการขายไม่มีผลต่อการตัดสินใจเลือกบริโภค',
       'อื่น ๆ'], dtype=object)

### MNlogit model

In [45]:
col_y = ['5. หากให้ท่านเลือกรับประทานบุฟเฟต์ชาบูได้ 1 แบบ\nท่านจะเลือกแบบใด']
# col_y =[ '1.3 เพศ','1.4 อายุ', '1.5 รายได้เฉลี่ยต่อเดือน', '1.6 อาชีพ' ,'2.1 ในเดือนที่ผ่านมาท่านทานบุฟเฟต์ชาบูกี่ครั้ง','2.2 เหตุผลใดที่ทำให้ท่านเลือกที่จะรับประทานบุฟเฟต์ชาบู'
#         ,'2.3 ส่วนใหญ่ท่านรับประทานบุฟเฟต์ชาบูกับใคร','2.4 โดยส่วนใหญ่ราคาบุฟเฟต์ชาบูต่อหัวที่ท่านเลือกรับประทานอยู่ที่เท่าไร']
col_x =[ '3.1 product_hat' ,'3.2 Price_hat' ,'3.3 Place_hat' ,'3.4 Promotion_hat' ,'3.5 People_hat' ,'3.6 process_hat']
col_x_dummy = ['1.3 เพศ','1.4 อายุ', '1.5 รายได้เฉลี่ยต่อเดือน', '1.6 อาชีพ' ]
y = df[col_y]
x = df[col_x]
x_dummy = df[col_x_dummy]
y = y[~y.isnull().any(axis=1)]
x = x[~x.isnull().any(axis=1)]
x_dummy = x_dummy[~x_dummy.isnull().any(axis=1)]
# y = new_df[col_x]
# x = new_df[col_y]
# for clean data
# y = pd.get_dummies(y,dtype=float)
# y.drop(['1.1 ท่านเคยรับประทานบุปเฟต์ชาบูหรือไม่_ไม่เคย'],axis=1,inplace=True)
# x_dummy = pd.get_dummies(x_dummy,dtype=float,drop_first=True)
x_dummy = pd.get_dummies(x_dummy,dtype=float)
x_dummy = x_dummy[['1.3 เพศ_หญิง', '1.4 อายุ_21 - 30 ปี','1.4 อายุ_31 - 40 ปี', '1.4 อายุ_40 ปีขึ้นไป','1.5 รายได้เฉลี่ยต่อเดือน_100,000 บาทขึ้นไป',
       '1.5 รายได้เฉลี่ยต่อเดือน_12,000 - 15,000 บาท','1.5 รายได้เฉลี่ยต่อเดือน_15,001 - 35,000 บาท','1.5 รายได้เฉลี่ยต่อเดือน_35,001 - 50,000 บาท',
       '1.5 รายได้เฉลี่ยต่อเดือน_50,001 - 100,000 บาท','1.6 อาชีพ_ข้าราชการ/รัฐวิสาหกิจ', '1.6 อาชีพ_พนักงานบริษัทเอกชน']]
x = pd.concat([x, x_dummy],axis = 1)
# x = sm.add_constant(x)
# x_dummy = sm.add_constant(x_dummy)
# MNlogit model
model = sm.MNLogit(y, x)
result = model.fit()
print(result.summary())

         Current function value: 1.308141
         Iterations: 35
                                                 MNLogit Regression Results                                                 
Dep. Variable:     5. หากให้ท่านเลือกรับประทานบุฟเฟต์ชาบูได้ 1 แบบ
ท่านจะเลือกแบบใด   No. Observations:                  176
Model:                                                                      MNLogit   Df Residuals:                      108
Method:                                                                         MLE   Df Model:                           64
Date:                                                              Sun, 28 Apr 2024   Pseudo R-squ.:                  0.1180
Time:                                                                      09:54:57   Log-Likelihood:                -230.23
converged:                                                                    False   LL-Null:                       -261.05
Covariance Type:                                           

c:\Anaconda 3\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [30]:
coef = pd.DataFrame(data=result.params)
coef.rename(columns = {0:'coef_B',1:'coef_C',2:'coef_D',3:'coef_E'}, inplace = True)
P_value = pd.DataFrame(data=result.pvalues)
P_value.rename(columns = {0:'P_value_B',1:'P_value_C',2:'P_value_D',3:'P_value_E'}, inplace = True)

In [33]:
all = pd.concat([coef, P_value],axis = 1)
all

,coef_B,coef_C,coef_D,coef_E,P_value_B,P_value_C,P_value_D,P_value_E
3.1 product_hat,0.299151,0.011528,-0.608755,-1.520344,0.585922,0.982663,0.265656,0.198474
3.2 Price_hat,-0.829825,-1.067119,-0.270343,-0.221887,0.068654,0.013509,0.550712,0.812211
3.3 Place_hat,-0.340909,-0.136162,-0.086279,0.160683,0.422734,0.737467,0.839204,0.833349
3.4 Promotion_hat,0.208769,0.414312,0.293959,-0.307920,0.564887,0.217830,0.405921,0.624398
3.5 People_hat,-0.151427,0.302882,0.144301,-0.123519,0.769013,0.582313,0.787213,0.907982
3.6 process_hat,0.495381,0.582130,0.331413,0.864377,0.415625,0.315024,0.585587,0.548357
1.3 เพศ_หญิง,-0.344007,-0.369574,0.049865,1.392952,0.545074,0.483337,0.928533,0.320634
1.4 อายุ_21 - 30 ปี,0.761540,-0.734775,0.105137,-2.100242,0.515827,0.473873,0.929269,0.266871
1.4 อายุ_31 - 40 ปี,1.338554,0.089579,0.996757,-2.448197,0.282283,0.934469,0.415375,0.256008
1.4 อายุ_40 ปีขึ้นไป,1.500938,0.506548,1.936910,-21.412472,0.338484,0.704109,0.175576,0.999444


In [328]:
all.to_excel('result.xlsx')

## olslogit

In [ ]:
df.groupby(['2.1 ในเดือนที่ผ่านมาท่านทานบุฟเฟต์ชาบูกี่ครั้ง']).size()

2.1 ในเดือนที่ผ่านมาท่านทานบุฟเฟต์ชาบูกี่ครั้ง
0     50
1    140
2     10
dtype: int64

In [ ]:
## define y,x variable
col_x = ['2.1 ในเดือนที่ผ่านมาท่านทานบุฟเฟต์ชาบูกี่ครั้ง']
# col_y =[ '1.3 เพศ','1.4 อายุ', '1.5 รายได้เฉลี่ยต่อเดือน', '1.6 อาชีพ' ,'2.1 ในเดือนที่ผ่านมาท่านทานบุฟเฟต์ชาบูกี่ครั้ง','2.2 เหตุผลใดที่ทำให้ท่านเลือกที่จะรับประทานบุฟเฟต์ชาบู'
#         ,'2.3 ส่วนใหญ่ท่านรับประทานบุฟเฟต์ชาบูกับใคร','2.4 โดยส่วนใหญ่ราคาบุฟเฟต์ชาบูต่อหัวที่ท่านเลือกรับประทานอยู่ที่เท่าไร']
col_y =[ '1.3 เพศ','1.4 อายุ', '1.5 รายได้เฉลี่ยต่อเดือน', '1.6 อาชีพ']
y = df[col_x]
x = df[col_y]
# y = new_df[col_x]
# x = new_df[col_y]
# for clean data
# y = pd.get_dummies(y,dtype=float)
# y.drop(['1.1 ท่านเคยรับประทานบุปเฟต์ชาบูหรือไม่_ไม่เคย'],axis=1,inplace=True)
x = pd.get_dummies(x,dtype=float,drop_first=True)
# MNlogit model
model = sm.Logit(y, x)
result = model.fit()
print(result.summary())

ValueError: endog must be in the unit interval.